<h2>Baseline Code</h2>

In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model, to_categorical
import warnings
import re

In [2]:
ls "."

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: BCA4-98AF

 C:\Users\inhoinno\DACON\2020_DACON-Kaggle\소설저자AI 디렉터리

2020-11-26  오후 07:31    <DIR>          .
2020-11-26  오후 07:31    <DIR>          ..
2020-11-25  오전 12:07    <DIR>          .ipynb_checkpoints
2020-11-24  오전 10:49            33,874 Augmentation.ipynb
2020-11-24  오전 11:15    <DIR>          dataset
2020-11-05  오후 05:31    <DIR>          ensemble
2020-10-09  오후 05:05         1,327,724 Logistic_regression.ipynb
2020-10-29  오후 06:12         3,959,848 NPL_base_5layer.h5
2020-11-12  오전 11:23            35,818 NPL_BaseLine.ipynb
2020-11-21  오후 11:47           992,363 NPL_BaseLine-CV.ipynb
2020-11-26  오후 07:31            54,108 NPL_BaseLine-CV-Copy1.ipynb
2020-11-05  오전 12:07           934,892 NPL_BaseLine-LSTM.ipynb
2020-11-12  오전 11:23           942,567 NPL_BaseLine-ONE-HOT.ipynb
2020-11-20  오후 05:49           115,342 NPL_Base-Tfidf-CV.ipynb
2020-11-20  오후 05:49           115,342 NPL_Base-Tfidf-CV-Copy1.ipynb
2020-11-12  오전 11:23           927,110 NP

In [3]:
#파일 불러오기
train = pd.read_csv('open/train_processed_500.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [4]:
train

,index,text,author,length
0,0,He was almost choking. There was so much so mu...,3,235
1,1,Your sister asked for it I suppose,2,34
2,2,She was engaged one day as she walked in peru...,1,312
3,3,The captain was in the porch keeping himself c...,4,305
4,4,Have mercy gentlemen odin flung up his hands. ...,3,215
...,...,...,...,...
183463,183463,To be urging her opinion against Sir odins wa...,1,571
183464,183464,After many tears an unwritten contract was dr...,3,536
183465,183465,They express the consciousness that you have ...,3,525
183466,183466,And of course he knows himself that he is doi...,3,515


In [5]:
train.author.describe()

count    183468.000000
mean          2.049088
std           1.285455
min           0.000000
25%           1.000000
50%           2.000000
75%           3.000000
max           4.000000
Name: author, dtype: float64

<h2>전처리

In [6]:
#부호를 제거해주는 함수

def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [7]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author,length
0,0,He was almost choking There was so much so muc...,3,235
1,1,Your sister asked for it I suppose,2,34
2,2,She was engaged one day as she walked in peru...,1,312
3,3,The captain was in the porch keeping himself c...,4,305
4,4,Have mercy gentlemen odin flung up his hands D...,3,215
...,...,...,...,...
183463,183463,To be urging her opinion against Sir odins wa...,1,571
183464,183464,After many tears an unwritten contract was dr...,3,536
183465,183465,They express the consciousness that you have ...,3,525
183466,183466,And of course he knows himself that he is doi...,3,515


1. 불용어 제거 remove_stopwords

In [8]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

2. One-Hot EnCoding

In [9]:
df = train

In [10]:
cat_cols = ['text']
print(f'categorical ({len(cat_cols)}):\t{cat_cols}')

categorical (1):	['text']


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

t = Tokenizer()
t.fit_on_texts(df['text'])
print(t.word_index) 
# 각 단어에 대한 인코딩 결과 출력.

{'the': 1, 'and': 2, 'of': 3, 'to': 4, 'a': 5, 'i': 6, 'in': 7, 'was': 8, 'he': 9, 'that': 10, 'it': 11, 'odin': 12, 'his': 13, 'you': 14, 'had': 15, 'with': 16, 'for': 17, 'her': 18, 'as': 19, 'at': 20, 'not': 21, 'but': 22, 'my': 23, 'she': 24, 'be': 25, 'have': 26, 'is': 27, 'him': 28, 'on': 29, 'me': 30, 'all': 31, 'so': 32, 'which': 33, 'by': 34, 'from': 35, 'were': 36, 'this': 37, 'there': 38, 'been': 39, 'they': 40, 'would': 41, 'no': 42, 'one': 43, 'said': 44, 'what': 45, 'an': 46, 'mr': 47, 'we': 48, 'could': 49, 'if': 50, 'very': 51, 'when': 52, 'or': 53, 'out': 54, 'are': 55, 'them': 56, 'upon': 57, 'up': 58, 'more': 59, 'who': 60, 'your': 61, 'will': 62, 'some': 63, 'now': 64, 'their': 65, 'man': 66, 'do': 67, 'into': 68, 'then': 69, 'only': 70, 'time': 71, 'little': 72, 'about': 73, 'such': 74, 'before': 75, 'did': 76, 'than': 77, 'any': 78, 'am': 79, 'know': 80, 'like': 81, 'its': 82, 'how': 83, 'should': 84, 'odins': 85, 'must': 86, 'see': 87, 'well': 88, 'our': 89, 'dow

from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

ohe = OneHotEncoder()
X = hstack((ohe.fit_transform(df[cat_cols])))
print(X.shape)

3. N-grams

4. Bag-of-words

In [12]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [13]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [14]:
train
#문체를 다 짤라먹었는데 왜 다 짤라먹었을까????
# , 활용 "" 활용같은것도 중요할것 같은데

,index,text,author,length
0,0,almost choking much much wanted say strange ex...,3,235
1,1,sister asked suppose,2,34
2,2,engaged one day walked perusing janes last let...,1,312
3,3,captain porch keeping carefully way treacherou...,4,305
4,4,mercy gentlemen odin flung hands dont write an...,3,215
...,...,...,...,...
183463,183463,urging opinion sir odins proof extremity case ...,1,571
183464,183464,many tears unwritten contract drawn us first n...,3,536
183465,183465,express consciousness no enemy punish pain con...,3,525
183466,183466,course knows no sort good moans knows better a...,3,515


<h2> Stratified K-Fold Cross Validation

In [15]:
n_fold = 5
n_class = 5
seed = 777


In [16]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

<h2> Keras 모델링

현재 코드는 Keras내부에서 바로 하는 Tokenizer 부분

In [17]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 2500
padding_type='post'
#oov_tok = "<OOV>"

In [18]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [19]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [20]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

<h3> 가벼운 NLP모델 생성

In [21]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [22]:
def get_base_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [23]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2500, 16)          320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


<h3> Stratified KFold NLP 모델 학습

In [24]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20


InternalError: 2 root error(s) found.
  (0) Internal:  Blas GEMM launch failed : a.shape=(32, 16), b.shape=(16, 24), m=32, n=24, k=16
	 [[node sequential/dense/MatMul (defined at <ipython-input-24-7b460eef7b03>:3) ]]
	 [[gradient_tape/sequential/embedding/embedding_lookup/Reshape/_40]]
  (1) Internal:  Blas GEMM launch failed : a.shape=(32, 16), b.shape=(16, 24), m=32, n=24, k=16
	 [[node sequential/dense/MatMul (defined at <ipython-input-24-7b460eef7b03>:3) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_786]

Function call stack:
train_function -> train_function


In [25]:
trn = train_padded
y = y_train
tst = test_padded
(trn.shape,y.shape,tst.shape)

((183468, 2500), (183468,), (19617, 2500))

In [26]:
tst

array([[   2,   37,    4, ...,    0,    0,    0],
       [   3,  229,  419, ...,    0,    0,    0],
       [ 119, 3384, 1153, ...,    0,    0,    0],
       ...,
       [   1,  371, 2124, ...,    0,    0,    0],
       [  90,    7,   15, ...,    0,    0,    0],
       [ 567, 5821,   10, ...,    0,    0,    0]])

In [ ]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    
    clf = get_base_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                            patience=3, min_lr=1e-6, mode='min', verbose=1)

    clf.fit(trn[i_trn], 
            y[i_trn],
            validation_data=(trn[i_val], (y[i_val])),
            epochs=100,
            callbacks=[es, rlr])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

<h3> Submission

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = p_tst
sample_submission

In [ ]:
sample_submission.to_csv('open/sub/submission_BASELINE_CV_processed2.csv', index = False, encoding = 'utf-8')
